# **Pymongo**

Vamos a probar la instalación de pymongo, librería que nos servirá para gestionar bases de datos de mongo desde nuestro entorno de programación más utilizado. Probaremos a ejecutar comandos básicos.

In [ ]:
# !pip install pymongo

### **Probamos comandos básicos de pymongo**

A continuación, vamos a definir una clase para crear una base de datos de estudiantes, dicha clase no es necesaria siempre pero sirve para facilitar la utilización de pymongo, en este caso cada método de la clase utiliza comandos básicos para hacer peticiones de MongoDB como: 

* **find()**
* **insert_one()**
* **find_one()**
* **update_one()**
* **delete_one()**

In [1]:
from pymongo import MongoClient

#Clase para crear una base de datos de estudiantes con MongoDB
class MongoStudentDatabase:
    def __init__(self, db_url, db_name):
        self.client = MongoClient(db_url) #Inicializa el cliente
        self.db = self.client[db_name] #Crea la base de datos
        self.students_collection = self.db['students'] #Crea la colección de estudiantes

    def add_student(self, student_data):
        result = self.students_collection.insert_one(student_data)
        return result.inserted_id

    def get_students(self):
        students = self.students_collection.find()
        return list(students)

    def get_student_by_id(self, student_id):
        student = self.students_collection.find_one({'_id': student_id})
        return student

    def update_student(self, student_id, new_data):
        result = self.students_collection.update_one({'_id': student_id}, {'$set': new_data})
        return result.modified_count

    def delete_student(self, student_id):
        result = self.students_collection.delete_one({'_id': student_id})
        return result.deleted_count


In [4]:
# Ejemplo de uso
if __name__ == '__main__':
    db_url = 'mongodb://localhost:27017/'
    db_name = 'data_science_class'
    student_db = MongoStudentDatabase(db_url, db_name)

    # Agregar un estudiante
    new_student_data = {
        'name': 'Juan Perez',
        'age': 22,
        'courses': ['Data Analysis', 'Machine Learning']
    }
    inserted_id = student_db.add_student(new_student_data)
    print(f'Student added with ID: {inserted_id}')

    # Obtener todos los estudiantes
    all_students = student_db.get_students()
    print('All Students:')
    for student in all_students:
        print(student)

    # Actualizar la información de un estudiante
    student_to_update = inserted_id
    update_data = {'age': 23}
    modified_count = student_db.update_student(student_to_update, update_data)
    print(f'Updated {modified_count} student(s)')

    # # Borrar un estudiante
    # student_to_delete = inserted_id
    # deleted_count = student_db.delete_student(student_to_delete)
    
    # print(f'Deleted {deleted_count} student(s)')

Student added with ID: 64f1bc303b112f25019ea9c9
All Students:
{'_id': ObjectId('64f1bc283b112f25019ea9c7'), 'name': 'Juan Perez', 'age': 23, 'courses': ['Data Analysis', 'Machine Learning']}
{'_id': ObjectId('64f1bc303b112f25019ea9c9'), 'name': 'Juan Perez', 'age': 22, 'courses': ['Data Analysis', 'Machine Learning']}
Updated 1 student(s)


### **ORM y ODM**

Un ***ODM (Object-Document Mapping)*** y un ***ORM (Object-Relational Mapping)*** son patrones de diseño utilizados en el desarrollo de software para mapear objetos de un lenguaje de programación a las estructuras de datos de una base de datos. Aunque ambos patrones tienen un propósito similar, están destinados a diferentes tipos de bases de datos y sistemas de almacenamiento.

<br>

**Diferencias:**

* Tipo de Base de Datos: ORM se utiliza principalmente para bases de datos relacionales, mientras que ODM se usa para bases de datos NoSQL, especialmente bases de datos orientadas a documentos.

* Estructura de Datos: ORM trabaja con tablas, filas y columnas, mientras que ODM trabaja con documentos, que pueden tener estructuras anidadas y variadas.

* Flexibilidad: ODM tiende a ser más flexible debido a la naturaleza variada de los documentos en bases de datos NoSQL. ORM sigue una estructura más rígida debido a las tablas y relaciones de las bases de datos relacionales.

<br>

**Ventajas del ORM:**

Abstrae la complejidad de las consultas SQL y permite trabajar con objetos en lugar de tablas y registros.
Facilita la creación, modificación y eliminación de registros en la base de datos a través de métodos en objetos.
Permite realizar consultas y filtrado de datos utilizando una sintaxis orientada a objetos.
Puede gestionar relaciones entre tablas y asociaciones entre objetos.
ODM (Object-Document Mapping):
Un ODM es similar a un ORM, pero se utiliza para mapear objetos de un lenguaje de programación a documentos en bases de datos NoSQL, específicamente en bases de datos orientadas a documentos como MongoDB. Los documentos en estas bases de datos son estructuras flexibles similares a JSON que pueden contener datos anidados y variados.

<br>

**Ventajas del ODM:**

Abstrae la complejidad de las operaciones de la base de datos NoSQL y permite trabajar con documentos como si fueran objetos.
Facilita la creación, modificación y eliminación de documentos en la base de datos a través de métodos en objetos.
Admite la flexibilidad de los documentos NoSQL y puede manejar estructuras de datos variadas.
Puede manejar relaciones entre documentos en bases de datos NoSQL que admiten estas relaciones.

### Con mongo engine

In [ ]:
# !python -m pip install -U mongoengine

In [6]:
from mongoengine import connect, Document, StringField, IntField, ListField, disconnect

class Student(Document):
    name = StringField(required=True)
    age = IntField()
    courses = ListField(StringField())

class MongoEngineStudentDatabase:
    def __init__(self, db_name):
        connect(db_name)

    def add_student(self, student_data):
        student = Student(**student_data)
        student.save()
        return student.id

    def get_students(self):
        students = Student.objects()
        return list(students)

    def get_student_by_id(self, student_id):
        student = Student.objects(id=student_id).first()
        return student

    def update_student(self, student_id, new_data):
        student = Student.objects(id=student_id).first()
        if student:
            for key, value in new_data.items():
                setattr(student, key, value)
            student.save()
            return 1
        return 0

    def delete_student(self, student_id):
        student = Student.objects(id=student_id).first()
        if student:
            student.delete()
            return 1
        return 0

In [7]:
# Ejemplo de uso
if __name__ == '__main__':
    db_name = 'data_science_class'
    student_db = MongoEngineStudentDatabase(db_name)

    # Agregar un estudiante
    new_student_data = {
        'name': 'Ana Lopez',
        'age': 25,
        'courses': ['Data Visualization', 'Python for Data Science']
    }
    inserted_id = student_db.add_student(new_student_data)
    print(f'Student added with ID: {inserted_id}')

    # Obtener todos los estudiantes
    all_students = student_db.get_students()
    print('All Students:')
    for student in all_students:
        print(student.to_json())

    # Actualizar la información de un estudiante
    student_to_update = inserted_id
    update_data = {'age': 26}
    modified_count = student_db.update_student(student_to_update, update_data)
    print(f'Updated {modified_count} student(s)')

    # Borrar un estudiante
    student_to_delete = inserted_id
    deleted_count = student_db.delete_student(student_to_delete)
    print(f'Deleted {deleted_count} student(s)')

Student added with ID: 64f1bd3d3b112f25019ea9cb
All Students:
{"_id": {"$oid": "64f1bd3d3b112f25019ea9cb"}, "name": "Ana Lopez", "age": 25, "courses": ["Data Visualization", "Python for Data Science"]}
Updated 1 student(s)
Deleted 1 student(s)


In [8]:
#Desconectarse de la base de datos
disconnect()

### Ventajas de mongoengine

MongoEngine es una biblioteca de mapeo de objetos (ODM, por sus siglas en inglés) para MongoDB en Python. Proporciona una abstracción de alto nivel sobre las operaciones de base de datos y ofrece varias ventajas:

- **Abstracción de Documentos**: MongoEngine permite a los desarrolladores trabajar con documentos de MongoDB de manera más similar a trabajar con objetos Python. Cada documento MongoDB se mapea a una clase Python, lo que facilita la interacción y el mantenimiento del código.

- **Sintaxis Más Amigable**: MongoEngine utiliza una sintaxis más orientada a objetos para definir esquemas de documentos y realizar operaciones CRUD en la base de datos. Esto hace que el código sea más legible y mantenible.

- **Validaciones Integradas**: Puedes definir validaciones directamente en los modelos MongoEngine, lo que garantiza que los documentos insertados cumplan con ciertos requisitos antes de ser almacenados en la base de datos.

- **Consultas Simplificadas**: Las consultas a la base de datos se pueden realizar utilizando una sintaxis similar a las consultas en MongoDB. Esto permite a los desarrolladores crear consultas complejas de manera más intuitiva.

- **Relaciones y Referencias**: MongoEngine admite relaciones entre documentos y permite referencias entre ellos, lo que facilita la construcción de modelos de datos más complejos.

- **Manejo de Herencia**: Puedes crear jerarquías de clases de modelos, lo que simplifica el manejo de documentos que comparten propiedades en común.

- **Migraciones de Esquema**: MongoEngine permite realizar migraciones de esquema más fácilmente. Puedes agregar, eliminar o modificar campos en tus modelos y luego aplicar estas modificaciones a la base de datos.

- **Integración con Frameworks Web**: MongoEngine se puede integrar con varios frameworks web populares, lo que facilita la creación de aplicaciones web basadas en MongoDB.

- **Adaptabilidad**: Puedes cambiar la base de datos subyacente entre diferentes instancias de MongoDB (local, Atlas, otros) sin cambiar demasiado el código, ya que la mayoría de las operaciones se realizan a través de la abstracción de MongoEngine.

- **Documentación Completa**: MongoEngine cuenta con una documentación completa y ejemplos útiles que facilitan el proceso de aprendizaje y desarrollo.

En general, MongoEngine es una opción poderosa para trabajar con MongoDB en Python, especialmente cuando se trabaja en proyectos que requieren un alto nivel de abstracción y facilitan la administración de esquemas y relaciones de datos.

### Ejemplos de uso de MongoEngine para establecer relaciones entre colecciones

In [124]:
#Ejemplo crear una base de datos -- La base de datos no se muestra hasta que no tiene al menos un documento
from mongoengine import connect

db_name = 'your_database_name'
db_connection = connect(db_name)


In [125]:
#Se crea la clase "Person" que hereda a su vez atributos de la clase "Document" proveniente de pyengine
from mongoengine import Document, StringField, IntField, connect, ReferenceField

class Person(Document):
    name = StringField(required=True) #Campo de string con un condicional ("Debe si o si ser string")
    age = IntField() #Campo integer


In [126]:
#Creamos un documento, ahora si debe mostrar la base de datos en MongoCompass por ejemplo
person = Person(name='Alice', age=30)
person.save()


<Person: Person object>

In [127]:
#Hacemos una query
adults = Person.objects(age__gt=25) #gte -- Greater Than-Equal
for adult in adults:
    print(f"Nombre: {adult.name}, Edad: {adult.age}")

Nombre: Alice, Edad: 30


#### A continuacion se muestran dos enfoques para establecer relaciones entre colecciones.

**Enfoque de Composición:** La primera definición de Student es adecuada si un estudiante está relacionado con una persona (instructor), pero no necesariamente tiene todos los mismos campos que una persona. Esto representa una relación de "composición" donde un estudiante contiene una referencia a una persona.

**Enfoque de Herencia:** La segunda definición de Student es adecuada si todos los estudiantes tienen los mismos campos que las personas y solo se diferencia por el campo student_id. Esto representa una relación "es un" (un estudiante es un tipo de persona).



In [128]:
#Primera definición
#Creamos una clase "Course" con un nombre del curso que tendrá de referencia una persona

# class Person(Document): --- Creada anteriormente
#     name = StringField(required=True) #Campo de string con un condicional ("Debe si o si ser string")
#     age = IntField() #Campo integer

class Course(Document):
    name = StringField(required=True)
    instructor = ReferenceField(Person)

In [130]:
# Creación de una instancia de Person (un instructor)
instructor = Person(name="Pepe instructor", age=30)
instructor.save()

# Creación de una instancia de Course con el instructor asignado
course = Course(name="Curso de MongoDB", instructor=instructor)
course.save()

# Recuperación de la información del curso con su instructor
retrieved_course = Course.objects(name="Curso de MongoDB").first()
print(f"Curso: {retrieved_course.name}")
print(f"Instructor: {retrieved_course.instructor.name}")


Curso: Curso de MongoDB
Instructor: Pepe instructor


In [134]:
#Segunda definición
#Creamos una clase "Course" con un instructor que tendrá los mismos campos que la clase "Person"

db_name = 'your_database_name_2'
db_connection = connect(db_name)

class Person(Document):
    name = StringField(required=True)
    age = IntField(min_value=1) #Especificamos condicion de edad minima igual a 1
    meta = {'allow_inheritance': True}#Permitimos la herencia

class Student(Person):
    student_id = StringField(required=True)

In [136]:
#Creamos un documento persona
person = Person(name='Alice', age=30)
person.save()


# Creación de una instancia de Student con un student_id
student = Student(name="Jose", age="20", student_id="S12345")
student.save()

# Recuperación de la información del estudiante
retrieved_student = Student.objects(student_id="S12345").first()
print(f"Nombre: {retrieved_student.name}")
print(f"Edad: {retrieved_student.age}")
print(f"ID de Estudiante: {retrieved_student.student_id}")

Nombre: Jose
Edad: 20
ID de Estudiante: S12345


In [74]:
# class Person(Document):
#     name = StringField(required=True)
#     age = IntField(min_value=1)
#     email = StringField() #Podemos agregar luego algún otro campo
#     direccion = StringField()


In [132]:
disconnect()